# Yield Curve & Term Structure Modeling

Full end-to-end workflow: ingestion, summary, fitting, performance evaluation, Monte Carlo scenarios, and scenario analysis.

In [ ]:
import sys
sys.path.append('../src')
import pandas as pd
import numpy as np
from data_ingest import load_from_csv
from model import fit_term_structure, monte_carlo_scenarios
from utils import year_fraction, plot_term_structure
import matplotlib.pyplot as plt

## Data Ingestion & Overview

In [ ]:
# Load data
df = load_from_csv('../data/combined_yields.csv', parse_dates=['date'])
# Preview
display(df.head())

### Summary statistics of raw yields

In [ ]:
print(df[['yield_1y','yield_2y','yield_5y','yield_10y']].describe())

## Fit Term Structure & Plot

In [ ]:
# Fit for latest date
today = df['date'].max()
slice_ = df[df['date']==today]
cs = fit_term_structure(slice_['date'], slice_['yield_10y'])
tenors = np.array([1,2,5,10])
# Plot
plot_term_structure(tenors, cs(tenors), title=f'Term Structure on {today.date()}')

## Performance Evaluation (Backtest)

In [ ]:
# Evaluate RMSE across history
def rmse(actual, pred):
    return np.sqrt(np.mean((actual - pred)**2))
rmses = []
for d, group in df.groupby('date'):
    cs_temp = fit_term_structure(group['date'], group['yield_10y'])
    preds = cs_temp(tenors)
    actual = group[['yield_1y','yield_2y','yield_5y','yield_10y']].values.flatten()
    # ensure matching lengths
    rmses.append(rmse(actual, np.tile(preds, len(group))))
print('Average RMSE across dates:', np.mean(rmses))

## Monte Carlo Scenario Generation

In [ ]:
# Generate scenarios
scenarios = monte_carlo_scenarios(cs, tenors, n_scenarios=500)
scenario_df = pd.DataFrame(scenarios, columns=tenors)
display(scenario_df.head())

### Scenario Summary Statistics

In [ ]:
summary = scenario_df.describe().T
display(summary[['mean','std','5%','95%']])

### Scenario Distribution Visualization

In [ ]:
plt.figure()
plt.hist(scenarios[:, -1], bins=30)
plt.title(f'Histogram of 10y Yield Scenarios on {today.date()}')
plt.xlabel('Yield')
plt.ylabel('Frequency')
plt.show()

## Save Outputs

In [ ]:
scenario_df.to_csv('../data/yield_scenarios_full.csv', index=False)
print('Scenarios saved to ../data/yield_scenarios_full.csv')